In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/5_AIADVANCE')

In [ ]:
# !unzip /content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats.zip -d /content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate, BatchNormalization, Dropout, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from imutils import paths
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Tiền xử lí dữ liệu

In [4]:
import os
# import shutil
# import random

# Đường dẫn tới thư mục gốc
# file_dir = '/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats'
root_dir = '/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/'

# Đường dẫn tới thư mục con 'train', 'val' và 'test'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

# # Tạo các thư mục con 'train', 'val' và 'test'
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Danh sách các nhãn
# labels = ['dog', 'cat']

# # Tỷ lệ train:val:test
# train_ratio = 0.7
# val_ratio = 0.2
# test_ratio = 0.1

# # Lặp qua các nhãn
# for label in labels:
#     # Tạo các thư mục con 'dog' và 'cat' trong 'train', 'val' và 'test'
#     os.makedirs(os.path.join(train_dir, label), exist_ok=True)
#     os.makedirs(os.path.join(val_dir, label), exist_ok=True)
#     os.makedirs(os.path.join(test_dir, label), exist_ok=True)

#     # Đường dẫn tới các tệp thuộc nhãn hiện tại
#     label_files = [file for file in os.listdir(file_dir) if file.startswith(label)]
#     print(label_files[0])
#     # Sắp xếp ngẫu nhiên các tệp
#     random.shuffle(label_files)

#     # Số lượng tệp trong từng thư mục con
#     num_files = len(label_files)
#     num_train = int(num_files * train_ratio)
#     num_val = int(num_files * val_ratio)
#     num_test = num_files - num_train - num_val

#     # Chia các tệp vào các thư mục con tương ứng
#     train_files = label_files[:num_train]
#     val_files = label_files[num_train:num_train + num_val]
#     test_files = label_files[num_train + num_val:]

#     for file in train_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(train_dir, label, file)
#         shutil.copy2(src, dst)

#     for file in val_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(val_dir, label, file)
#         shutil.copy2(src, dst)

#     for file in test_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(test_dir, label, file)
#         print(src)
#         print(dst)
#         shutil.copy2(src, dst)

## Cur

In [10]:
img_size = (224 , 224)
batch_size = 8
img_shape = (img_size[0] , img_size[1] , 3)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_directory(train_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

valid_gen = ts_gen.flow_from_directory(val_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

test_gen = ts_gen.flow_from_directory(test_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = False , batch_size = batch_size)

Found 3499 images belonging to 2 classes.
Found 892 images belonging to 2 classes.
Found 244 images belonging to 2 classes.


In [ ]:
# # Lấy các đường dẫn đến ảnh.
# # image_path = list(paths.list_images('data/dogs-vs-cats/train/train'))
# image_path = list(paths.list_images('/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats'))

# # Đổi vị trí ngẫu nhiên các đường dẫn ảnh
# random.shuffle(image_path)

# # Chỉ lấy 5000 dòng đầu tiên (5000 ảnh)
# # image_path = image_path[:5000]
# # print(image_path)

In [ ]:
# labels = [p.split(os.path.sep)[-1] for p in image_path]
# labels = [label.split('.')[-3] for label in labels]
# print(labels)
# # Chuyển tên các loài hoa thành số
# le = LabelEncoder()
# labels = le.fit_transform(labels)
# print(labels)

['dog', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'cat', 'cat'

In [ ]:
# # Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
# list_image = []
# for (j, imagePath) in enumerate(image_path):
#     # Load ảnh và resize về kích thước (224, 224)
#     image = load_img(imagePath, target_size=(224, 224))

#     # Chuyển ảnh thành mảng numpy
#     image = img_to_array(image)

#     # Mở rộng chiều của mảng numpy
#     image = np.expand_dims(image, 0)

#     # Tiền xử lý ảnh theo tiêu chuẩn của mô hình VGG16
#     image = imagenet_utils.preprocess_input(image)

#     # Thêm ảnh đã tiền xử lý vào danh sách
#     list_image.append(image)

# # Chuyển danh sách các ảnh thành mảng numpy một chiều
# list_image = np.vstack(list_image)

In [ ]:
# print(list_image.shape)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

#### Xây dựng Model

In [11]:
'''
MobileNet in TensorFlow2.

Reference:
[1] Sandler, Mark, et al.
    "Mobilenetv2: Inverted residuals and linear bottlenecks."
    Proceedings of the IEEE conference on computer vision and pattern recognition. 2018.
'''
import tensorflow as tf
from tensorflow.keras import layers

class Block(tf.keras.Model):
    '''Expand + depthwise & pointwise convolution'''
    def __init__(self, in_channels, out_channels, expansion, strides):
        super(Block, self).__init__()
        self.strides = strides
        channels = expansion * in_channels

        self.conv1 = layers.Conv2D(channels, kernel_size=1, use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(channels, kernel_size=3, strides=strides, padding='same',
                                   groups=channels, use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv2D(out_channels, kernel_size=1, use_bias=False)
        self.bn3 = layers.BatchNormalization()

        if strides == 1 and in_channels != out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(out_channels, kernel_size=1, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = tf.keras.activations.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = layers.add([self.shortcut(x), out]) if self.strides==1 else out
        return out

class MobileNetV2(tf.keras.Model):
    # (expansion, out_channels, num_blocks, strides)
    config = [(1, 16, 1, 1),
              (6, 24, 2, 1),  # NOTE: change strides 2 -> 1 for CIFAR10
              (6, 32, 3, 2),
              (6, 64, 4, 2),
              (6, 96, 3, 1),
              (6, 160, 3, 2),
              (6, 320, 1, 1)]

    def __init__(self, num_classes):
        super(MobileNetV2, self).__init__()
        # NOTE: change conv1 strides 2 -> 1 for CIFAR10
        self.conv1 = layers.Conv2D(32, kernel_size=3, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.layer = self._make_layers(in_channels=32)
        self.conv2 = layers.Conv2D(1280, kernel_size=1, use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.avg_pool2d = layers.AveragePooling2D(pool_size=4)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.layer(out)
        out = tf.keras.activations.relu(self.bn2(self.conv2(out)))
        out = self.avg_pool2d(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

    def _make_layers(self, in_channels):
        layer = []
        for expansion, out_channels, num_blocks, strides in self.config:
            stride = [strides] + [1]*(num_blocks-1)
            for s in stride:
                layer += [Block(in_channels, out_channels, expansion, s)]
                in_channels = out_channels
        return tf.keras.Sequential(layer)

In [12]:
mobileNetV2 = MobileNetV2(num_classes = 2)

In [13]:
mobileNetV2.build((1, 224, 224, 3))
mobileNetV2.summary()

Model: "mobile_net_v2_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_57 (Conv2D)          multiple                  864       
                                                                 
 batch_normalization_57 (Ba  multiple                  128       
 tchNormalization)                                               
                                                                 
 sequential_9 (Sequential)   (1, 28, 28, 320)          1903488   
                                                                 
 conv2d_113 (Conv2D)         multiple                  409600    
                                                                 
 batch_normalization_113 (B  multiple                  5120      
 atchNormalization)                                              
                                                                 
 average_pooling2d_1 (Avera  multiple              

#### Fit model

In [ ]:
# Tạo đối tượng optimizer  # SGD
optimizer = tf.keras.optimizers.Adam()

# Thiết lập ModelCheckpoint
save_best_only = True
checkpoint = ModelCheckpoint('models/model-MobileNetV2-{epoch:03d}.keras',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=save_best_only,
                             mode='auto')

# Xây dựng quy trình huấn luyện
mobileNetV2.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình với dữ liệu đã chuẩn bị
# H = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2, callbacks=[checkpoint])
H = mobileNetV2.fit(
    train_gen,
    steps_per_epoch = train_gen.samples // train_gen.batch_size,
    validation_data = valid_gen,
    validation_steps = valid_gen.samples // valid_gen.batch_size,
    epochs = 20,
    callbacks=[checkpoint]
)

Epoch 1/20
437/437 [==============================] - ETA: 0s - loss: 1.2024 - accuracy: 0.6184
Epoch 1: val_loss improved from inf to 0.79339, saving model to models/model-MobileNetV2-001.keras
437/437 [==============================] - 680s 1s/step - loss: 1.2024 - accuracy: 0.6184 - val_loss: 0.7934 - val_accuracy: 0.4561
Epoch 2/20
437/437 [==============================] - ETA: 0s - loss: 0.5888 - accuracy: 0.6918
Epoch 2: val_loss improved from 0.79339 to 0.63643, saving model to models/model-MobileNetV2-002.keras
437/437 [==============================] - 218s 498ms/step - loss: 0.5888 - accuracy: 0.6918 - val_loss: 0.6364 - val_accuracy: 0.6486
Epoch 3/20
437/437 [==============================] - ETA: 0s - loss: 0.5476 - accuracy: 0.7282
Epoch 3: val_loss did not improve from 0.63643
437/437 [==============================] - 217s 496ms/step - loss: 0.5476 - accuracy: 0.7282 - val_loss: 0.7317 - val_accuracy: 0.6396
Epoch 4/20
437/437 [==============================] - ETA: 0s

In [ ]:
# Vẽ đồ thị loss, accuracy của traning set và validation set
fig = plt.figure()
numOfEpoch = 10
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), H.history['accuracy'], label='accuracy') # sử dụng từ khóa accuracy thay vì acc
plt.plot(np.arange(0, numOfEpoch), H.history['val_accuracy'], label='validation accuracy') # sử dụng từ khóa val_accuracy thay vì val_acc
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()

In [ ]:
from keras.models import load_model
model = load_model('models/model-ResNet50-009.keras')

score = model.evaluate(X_test, y_test, verbose=1)
print(score)

In [ ]:
model.summary()

In [ ]:
# Đánh giá model
preds = model.predict(X_test) # DỰ ĐOÁN VÀ ĐÁNH GIÁ
preds = np.argmax(preds, axis=1)
print(classification_report(y_test, preds))

In [ ]:
# 10. Dự đoán ảnh
plt.imshow(X_test[0])
plt.axis('off')
plt.show()

y_predict = model.predict(np.expand_dims(X_test[0], axis=0))
print('Giá trị dự đoán: ', np.argmax(y_predict))

In [ ]:
# Tạo ma trận nhầm lẫn
cm = confusion_matrix(y_test, preds)

# Chuyển nhãn số thành tên
label_names = le.inverse_transform(np.unique(y_test))
label_names = [label.split('/')[-1] for label in label_names]
# Sắp xếp lại ma trận nhầm lẫn
sorted_cm = cm[np.argsort(label_names)][:, np.argsort(label_names)]

# Tạo dataframe từ ma trận nhầm lẫn đã sắp xếp lại
df_cm = pd.DataFrame(sorted_cm, index=label_names, columns=label_names)

# Vẽ ma trận nhầm lẫn
plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()
